In [17]:
import asyncio
import time
import re
import json

from indy import crypto, did, wallet

In [2]:
me = 'woshi'
wallet_config = '{"id": "%s-wallet"}' % me
wallet_credentials = '{"key": "%s-wallet-key"}' % me
await wallet.create_wallet(wallet_config, wallet_credentials)

In [3]:
wallet_handle = await wallet.open_wallet(wallet_config, wallet_credentials)
print('wallet = %s' % wallet_handle)

wallet = 3


In [4]:
(my_did, my_vk) = await did.create_and_store_my_did(wallet_handle, "{}")
print('my_did and verkey = %s %s' % (my_did, my_vk))

my_did and verkey = KS5k9WwiFy4ju2DVmzuisK B3gnR2fHKG4YUjwK6bQgHFxP9sRT7hNWivPoG7zZ2Qne


In [6]:
await did.key_for_local_did(wallet_handle, my_did)

'B3gnR2fHKG4YUjwK6bQgHFxP9sRT7hNWivPoG7zZ2Qne'

In [7]:
await did.list_my_dids_with_meta(wallet_handle)

'[{"did":"KS5k9WwiFy4ju2DVmzuisK","verkey":"B3gnR2fHKG4YUjwK6bQgHFxP9sRT7hNWivPoG7zZ2Qne","tempVerkey":null,"metadata":null}]'

In [8]:
##########################################################

In [10]:
their = 'Alice'
their_wallet_config = '{"id": "%s-wallet"}' % their
their_wallet_credentials = '{"key": "%s-wallet-key"}' % their
their_wallet_handle = await wallet.open_wallet(their_wallet_config, their_wallet_credentials)
print('their wallet = %s' % their_wallet_handle)

their wallet = 3


In [21]:
result = await did.list_my_dids_with_meta(their_wallet_handle)
(their_did, their_vk) = json.loads(result)[0]['did'], json.loads(result)[0]['verkey']
print('their_did and their_vk = %s %s' % (their_did, their_vk))

their_did and their_vk = AmERTwzGvXa78wLhBhgSAd 6Kch61WKhjHrLBrjak5FKmnRP3e3wkMJ3VogpgZRQaT8


In [23]:
##########################################################

In [24]:
msg = "hello world"

In [27]:
encrypted_msg = await crypto.crypto_sign(wallet_handle, my_vk, msg)
print(encrypted_msg)

b'\xfe\xa1\xe0u\xea`z>\x00\xbe6\xa4J\xd6\xfd\xb7\xe7\xf0\xe62\x97j\r\xd1)\xa3\x06\x00\xb6.\xf4\xa7\xa6\xe4\xb5c\xfe\xfb\x97F\xed\x12\x1e\x1eq\x8f\xe7$\xac\t\xd9]\x95\x9c\xb3\xc9\xe6I{\x7f\x92\xc8\x9c\t'


In [28]:
await crypto.crypto_verify(my_vk, msg, encrypted_msg)

True

In [38]:
##########################################################

In [35]:
challenge = await crypto.pack_message(wallet_handle, msg, [their_vk], None)
print(challenge)

b'{"protected":"eyJlbmMiOiJ4Y2hhY2hhMjBwb2x5MTMwNV9pZXRmIiwidHlwIjoiSldNLzEuMCIsImFsZyI6IkFub25jcnlwdCIsInJlY2lwaWVudHMiOlt7ImVuY3J5cHRlZF9rZXkiOiItN3VaMFhHLWkzNXhDNjY3TUxfQUZqX3FWb2JWcE9TbEd3cWVGMDNFaUFjcV92YmJTMEJFbEFRS2owUExQZkpSV1ZQUHBqMlJxelhwWVdJaVd0S0tfM2NXLUUwUU5UYWdSRVlvQ05rMG9TTT0iLCJoZWFkZXIiOnsia2lkIjoiNktjaDYxV0toakhyTEJyamFrNUZLbW5SUDNlM3drTUozVm9ncGdaUlFhVDgifX1dfQ==","iv":"Hw9hiBHhujuNSsYC","ciphertext":"ykde-tgy-bLMkdw=","tag":"f2TZ4BR2a78KcF6rRil04g=="}'


In [36]:
response = await crypto.unpack_message(their_wallet_handle, challenge)

In [37]:
print(response)

b'{"message":"hello world","recipient_verkey":"6Kch61WKhjHrLBrjak5FKmnRP3e3wkMJ3VogpgZRQaT8"}'


In [50]:
##########################################################

In [58]:
print(msg)

hello world


In [78]:
msg.encode('utf-8')

b'hello world'

In [79]:
challenge = await crypto.anon_crypt(their_vk, msg.encode('utf-8'))

In [80]:
print(challenge)

b'\x8d\xfe\x1e\xe8c\xde;\xbd\x99\x0e\x17Y\x10-\xb9\x02f\xd2\x8d\xae\x86\xbc\xf9Y\x89\x04"LcVU$\xf4\xc9_\xc3\xb3\xb2r\x99\xc3i\xeb\xc2a\xa4\x806\xf7\x199$b\x9d\xed\xb2v\xf5\xde'


In [81]:
response = await crypto.anon_decrypt(their_wallet_handle, their_vk, challenge)

In [82]:
print(response)

b'hello world'


In [84]:
response.decode('utf-8')

'hello world'